# **Detection of Hate Speech in Twitter Using an NLP Approach**

## Import Libraries

In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
!pip install emoji

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import transformers
import emoji as emoji
import re
from torch.utils.data import TensorDataset
from urllib.parse import urlparse
import string
import torch
import gc
import torch.nn as nn
from transformers import AutoModel
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification

In [ ]:
# specify GPU
device = torch.device("cuda")

Load & Visualize the Strongly Classified Tweets Dataset

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/HS_Data.csv') 

In [ ]:
count_df=df.groupby('class').count().reset_index()
count_df

,class,tweet
0,0,263
1,1,14347
2,2,2872


In [ ]:
fig = px.bar(count_df, x="class", y='tweet',title="Class Count")
fig.update_layout(width=500, height=400)
fig.show()

In [ ]:
fig_pie = px.pie(count_df, values='tweet', names='class',width=500, height=400)
fig_pie.show()

In [ ]:
df= df.reindex(columns=['tweet','class'])
df= df.rename(columns={'class': 'label'})
df

,tweet,label
0,I hate when hoes try to play the wifey roll,1
1,For guys ?bros over hoes?. For girls ?chicks o...,1
2,"In the words of lil Debbie ""I'm that bitch""",1
3,Talked to Deigo last night my dawg good like t...,1
4,"RT @MensHumor: Monday, what's up you ugly bitch?",1
...,...,...
17477,I got some bitch ass coworkers. I stay late ev...,1
17478,This bitch just chillin http://t.co/lp9Xd7qRzP,1
17479,Don't worry about the bitches I use to fuck wit.,1
17480,RT @40oz_VAN: Don't be that frontin' ass bitch.,1


In [ ]:
'''

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df=df.groupby([ 'label', 'data_type']).count().reset_index().rename(columns={'tweet':'Total'})
df
'''

"\n\nX_train, X_val, y_train, y_val = train_test_split(df.index.values, \n                                                  df.label.values, \n                                                  test_size=0.15, \n                                                  random_state=42, \n                                                  stratify=df.label.values)\n\ndf['data_type'] = ['not_set']*df.shape[0]\n\ndf.loc[X_train, 'data_type'] = 'train'\ndf.loc[X_val, 'data_type'] = 'val'\n\ndf=df.groupby([ 'label', 'data_type']).count().reset_index().rename(columns={'tweet':'Total'})\ndf\n"

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['tweet'], df['label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['label'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 280,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 280,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 280,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



In [ ]:
# for train set
train_input_ids = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_labels = torch.tensor(train_labels.tolist())

# for validation set
val_input_ids  = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_labels = torch.tensor(val_labels.tolist())

test_input_ids  = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_labels = torch.tensor(test_labels.tolist())


In [ ]:
dataset_train = TensorDataset(train_input_ids , train_mask, train_labels)
dataset_val = TensorDataset(val_input_ids, val_mask, val_labels)
dataset_test = TensorDataset(test_input_ids, test_mask,test_labels)
len(dataset_train), len(dataset_val),len(dataset_test)

(12237, 2622, 2623)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels= df['label'].nunique(),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test,
                                    sampler=SequentialSampler(dataset_test),                
                                    batch_size=batch_size)
                            

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),lr=1e-5,eps=1e-8)

In [ ]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    total_correct = 0
    total = 0
    for label in np.unique(labels_flat):
      y_preds = preds_flat[labels_flat==label]
      y_true = labels_flat[labels_flat==label]
      a = len(y_preds[y_preds==label])
      b = len(y_true)
      total_correct += a
      total += b
      #print(f'Class: {label_dict_inverse[label]}')
      print(f'Accuracy:{a}/{b}\n')
      print(f'Overall accuracy: {total_correct/total:.2f}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/4079 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.1734526235809407
Validation loss: 0.12154126572568202
F1 Score (Weighted): 0.9704018270050038


Epoch 2:   0%|          | 0/4079 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.09953919044699198
Validation loss: 0.12338011544427231
F1 Score (Weighted): 0.9718098985417138


Epoch 3:   0%|          | 0/4079 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.06030684906182463
Validation loss: 0.0990033813992607
F1 Score (Weighted): 0.9811562085897828


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)
print('predictions:', predictions)
print('true_vals:', true_vals)


predictions: [[-2.5533261  7.0633245 -4.3671827]
 [-1.1793535 -3.389599   5.0465307]
 [-2.3090646  7.044246  -4.6967454]
 ...
 [-2.4745293  6.8504043 -4.4667726]
 [-2.365078   7.00902   -4.6215925]
 [-1.8990368  6.6649404 -4.8482213]]
true_vals: [1 2 1 ... 1 1 1]


In [ ]:
from sklearn.metrics import accuracy_score,classification_report
print("Performance:")
# model's performance
print('Classification Report')
preds = np.argmax(predictions, axis=1)
print(classification_report(true_vals, preds))
print("Accuracy of BERT is " + str(accuracy_score(true_vals, preds)))


Performance:
Classification Report
              precision    recall  f1-score   support

           0       0.60      0.64      0.62        39
           1       0.99      0.99      0.99      2152
           2       0.97      0.97      0.97       431

    accuracy                           0.98      2622
   macro avg       0.85      0.87      0.86      2622
weighted avg       0.98      0.98      0.98      2622

Accuracy of BERT is 0.9809305873379099


In [ ]:
accuracy_per_class(predictions, true_vals)

Accuracy:25/39

Overall accuracy: 0.64

Accuracy:2128/2152

Overall accuracy: 0.98

Accuracy:419/431

Overall accuracy: 0.98

